# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [46]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [47]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [48]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [49]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [50]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [51]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [52]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [53]:
import numpy as np

def jacobi_iteration(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)  # Initialize the solution vector

    for _ in range(num_iterations):
        x_new = np.zeros(n)  # Initialize a new solution vector
        
        for i in range(n):
            sum_term = np.dot(A[i, :i], x[:i]) + np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - sum_term) / A[i, i]

        x = x_new

    return x



In [54]:
jacobi_iteration(A,b,n_iter)

array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
       -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648])

In [55]:
def jacobi_iterationb(A, b, num_iterations, tolerance):
    n = len(b)
    x = np.zeros(n)  # Initialize the solution vector

    for k in range(num_iterations):
        x_new = np.zeros(n)  # Initialize a new solution vector
        
        for i in range(n):
            sum_term = np.dot(A[i, :i], x[:i]) + np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - sum_term) / A[i, i]

        norm_diff = np.linalg.norm(x_new - x)
        if norm_diff < tolerance:
            print("Convergence achieved after", k+1, "iterations.")
            break

        x = x_new

    return x

In [56]:
jacobi_iterationb(A,b,n_iter,0.1)

Convergence achieved after 2 iterations.


array([0.05049637, 0.04568193, 0.05285884, 0.03683105, 0.06193394,
       0.0092481 , 0.00191869, 0.03822729, 0.00729613, 0.06067932])

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [57]:
import numpy as np

def seidel_iteration(A, b, n_iter):
    n = len(A)
    x = np.zeros(n)  # Start with an initial guess of zeros

    for _ in range(n_iter):
        for i in range(n):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    residual = A @ x - b
    norm_B = np.linalg.norm(A - np.diag(np.diag(A)))

    return x, residual, norm_B


In [58]:
# Generate a random matrix A
n = 10  # Size of the matrix
A = np.random.rand(n, n)

# Generate a random vector b
b = np.random.rand(n)

# Set the number of iterations
num_iterations = 100

# Perform the Seidel's iteration
solution = seidel_iteration(A, b, num_iterations)

print("Solution:", solution)


Solution: (array([-8.65306641e+212, -3.43096644e+212,  5.53508407e+212,
        1.32988064e+214, -2.35404848e+214,  1.17067683e+214,
        8.66891814e+213, -1.33380176e+215,  5.63558624e+214,
        2.22071187e+215]), array([ 3.58086455e+214,  1.23062869e+215,  5.18067531e+214,
       -1.15262433e+214, -1.27434528e+214,  1.76832416e+215,
        3.88964134e+214,  9.93394154e+214,  5.02575063e+214,
       -9.61363327e-001]), 5.543640625227805)


In [59]:
# Calculate the iteration matrix B
D = np.diag(np.diag(A))
L = np.tril(A, k=-1)
U = np.triu(A, k=1)
B = -np.linalg.inv(D).dot(L + U)

# Calculate the norm of B
norm_B = np.linalg.norm(B)

print("Norm of B:", norm_B)


Norm of B: 156.22351982913008


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [60]:
import numpy as np

def minimum_residual_scheme(A, b, num_iterations):
    n = len(b)
    x = np.zeros(n)  # Initialize the solution vector

    for _ in range(num_iterations):
        r = b - A.dot(x)  # Compute the residual vector
        alpha = np.dot(r, r) / np.dot(r, A.dot(r))  # Compute the iteration parameter
        x = x + alpha * r  # Update the solution vector

    return x


In [61]:
# Generate a random matrix A
n = 10  # Size of the matrix
A = np.random.rand(n, n)

# Generate a random vector b
b = np.random.rand(n)

# Set the number of iterations
num_iterations = 100

# Perform the minimum residual scheme
solution = minimum_residual_scheme(A, b, num_iterations)

print("Solution:", solution)


Solution: [-2.16903915e+09  1.22110991e+10 -5.40985667e+09  6.40129712e+09
 -2.67896427e+09 -5.69572792e+09  1.18495922e+09 -8.29484515e+09
  3.45888568e+09 -4.64386759e+09]


In [62]:
# Calculate the ground truth solution using a direct method
ground_truth_solution = np.linalg.solve(A, b)

# Calculate the residual
residual = b - A.dot(solution)

# Calculate the norm of the residual
norm_residual = np.linalg.norm(residual)

# Calculate the deviation from the ground truth solution
deviation = np.linalg.norm(solution - ground_truth_solution)

print("Norm of the residual:", norm_residual)
print("Deviation from the ground truth solution:", deviation)


Norm of the residual: 16332538039.100218
Deviation from the ground truth solution: 19168071685.51171
